# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/US_airports.gml")

In [3]:
print(nx.info(g))

Graph with 500 nodes and 2980 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 500 nodes and 2980 edges
Graph with 500 nodes and 2980 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/US_airports-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/US_airports-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/US_airports-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/US_airports-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

500

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[100,
 0,
 93,
 37,
 36,
 40,
 31,
 110,
 62,
 133,
 108,
 107,
 25,
 44,
 84,
 61,
 72,
 125,
 71,
 80,
 18,
 14,
 74,
 129,
 24,
 78,
 35,
 126,
 82,
 138,
 13,
 88,
 194,
 47,
 104,
 70,
 87,
 115,
 120,
 65,
 94,
 2,
 5,
 96,
 26,
 128,
 10,
 6,
 119,
 124,
 69,
 7,
 57,
 130,
 121,
 17,
 161,
 113,
 11,
 131,
 103,
 101,
 149,
 135,
 34,
 118,
 97,
 58,
 99,
 98,
 86,
 227,
 39,
 140,
 116,
 54,
 55,
 75,
 42,
 142,
 173,
 141,
 81,
 202,
 53,
 170,
 27,
 23,
 114,
 19,
 1,
 66,
 112,
 73,
 346,
 150,
 287,
 158,
 33,
 413,
 403,
 95,
 41,
 414,
 29,
 63,
 122,
 136,
 273,
 338,
 289,
 132,
 277,
 183,
 151,
 59,
 111,
 68,
 117,
 350,
 188,
 90,
 330,
 339,
 50,
 340,
 216,
 255,
 164,
 175,
 49,
 200,
 145,
 415,
 331,
 20,
 22,
 374,
 274,
 211,
 123,
 109,
 12,
 147,
 159,
 197,
 137,
 290,
 344,
 45,
 106,
 352,
 213,
 329,
 257,
 221,
 67,
 169,
 163,
 181,
 193,
 16,
 144,
 91,
 43,
 410,
 8,
 377,
 32,
 102,
 294,
 186,
 209,
 160,
 153,
 399,
 166,
 171,
 105,
 139,
 127,

In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{100: 500,
 0: 499,
 93: 498,
 37: 497,
 36: 496,
 40: 495,
 31: 494,
 110: 493,
 62: 492,
 133: 491,
 108: 490,
 107: 489,
 25: 488,
 44: 487,
 84: 486,
 61: 485,
 72: 484,
 125: 483,
 71: 482,
 80: 481,
 18: 480,
 14: 479,
 74: 478,
 129: 477,
 24: 476,
 78: 475,
 35: 474,
 126: 473,
 82: 472,
 138: 471,
 13: 470,
 88: 469,
 194: 468,
 47: 467,
 104: 466,
 70: 465,
 87: 464,
 115: 463,
 120: 462,
 65: 461,
 94: 460,
 2: 459,
 5: 458,
 96: 457,
 26: 456,
 128: 455,
 10: 454,
 6: 453,
 119: 452,
 124: 451,
 69: 450,
 7: 449,
 57: 448,
 130: 447,
 121: 446,
 17: 445,
 161: 444,
 113: 443,
 11: 442,
 131: 441,
 103: 440,
 101: 439,
 149: 438,
 135: 437,
 34: 436,
 118: 435,
 97: 434,
 58: 433,
 99: 432,
 98: 431,
 86: 430,
 227: 429,
 39: 428,
 140: 427,
 116: 426,
 54: 425,
 55: 424,
 75: 423,
 42: 422,
 142: 421,
 173: 420,
 141: 419,
 81: 418,
 202: 417,
 53: 416,
 170: 415,
 27: 414,
 23: 413,
 114: 412,
 19: 411,
 1: 410,
 66: 409,
 112: 408,
 73: 407,
 346: 406,
 150: 405,
 287: 40

In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/Infra-USAirports/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/Infra-USAirports/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True